In [1]:
import pandas as pd

from asbuilt import check_asbuilt

df_vins = pd.read_csv('vins.csv', dtype={'pr': 'str'})
df_vins_csv = df_vins.copy()

duplicates = df_vins[df_vins.duplicated(subset=['vin'], keep=False)]
print(f'Found {len(duplicates)} duplicate VINs')
if len(duplicates):
  print(duplicates)
  raise Exception('Duplicate VINs found')

# remove rows with non-empty 'pr' column (these were added in openpilot PRs)
df_vins = df_vins[df_vins['pr'].isnull()]
df_vins.drop(columns=['pr'], inplace=True)

# remove rows without 'Explorer' in the 'comment' column
# df_vins = df_vins[df_vins['comment'].str.contains('Explorer')]

# reset index
df_vins.reset_index(drop=True, inplace=True)

print(f'Loaded {len(df_vins)} VINs')
vins = list(df_vins['vin'])

check_asbuilt(vins)

Found 0 duplicate VINs
Loaded 845 VINs
Found AsBuilt data for 845 VINs


/tmp/ipykernel_115442/4021567364.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from asbuilt import AsBuiltData
from ecu import FordEcu
from settings import VehicleSettings

for vin in list(vins)[:3]:
  abd = AsBuiltData.from_vin(vin)

  print(f'{vin=}')

  if FordEcu.ImageProcessingModuleA not in abd.ecus:
    print('No IPMA data')
    print()
    print()
    continue

  # print(f'ecus={list(abd.ecus.keys())}')
  print('ecu=ImageProcessingModuleA')
  print(f'  identifiers={abd.get_identifiers(FordEcu.ImageProcessingModuleA)}')
  # for label, data in abd.get_configuration(FordEcu.ImageProcessingModuleA).items():
  #   if label == '01-03' or label == '02-01': break
  #   dataStr = ''.join(format(x, '02x') for x in data)
  #   print(f'    706-{label}: {dataStr[0:4]} {dataStr[4:8]} {dataStr[8:10]}')

  setting = VehicleSettings.ipma_enable_adaptive_cruise
  print(f'  setting="{setting.comment}" value={abd.get_setting_value(setting)} ({hex(abd.get_setting_data(setting))})')

  setting = VehicleSettings.ipma_enable_traffic_jam_assist
  print(f'  setting="{setting.comment}" value={abd.get_setting_value(setting)} ({hex(abd.get_setting_data(setting))})')

  print()
  print()

vin='1FMSK8DH1NGA52944'
ecu=ImageProcessingModuleA
  identifiers={61712: 'DSLB5T-19H406-AE', 61713: 'LB5T-14F403-CA', 61715: 'LB5T-19H406-CF', 61728: 'LB5T-14F397-BC', 61732: 'LB5T-14F398-AF', 61733: 'LB5T-14F398-BC', 61832: 'LB5T-14F397-AF', 61836: '220342335'}
  setting="Enable ACC" value=Off (0x4)
  setting="Enable TJA" value=Off (0x1)


vin='1FMSK8DH7MGC46036'
ecu=ImageProcessingModuleA
  identifiers={61712: 'DSLB5T-19H406-AE', 61713: 'LB5T-14F403-CA', 61715: 'LB5T-19H406-CF', 61728: 'LB5T-14F397-BC', 61732: 'LB5T-14F398-AF', 61733: 'LB5T-14F398-BC', 61832: 'LB5T-14F397-AF', 61836: '212752286'}
  setting="Enable ACC" value=Off (0x4)
  setting="Enable TJA" value=Off (0x1)


vin='1FMSK8DH9MGB42406'
ecu=ImageProcessingModuleA
  identifiers={61712: 'DSLB5T-19H406-AE', 61713: 'LB5T-14F403-CA', 61715: 'LB5T-19H406-CF', 61728: 'LB5T-14F397-BC', 61732: 'LB5T-14F398-AF', 61733: 'LB5T-14F398-BC', 61832: 'LB5T-14F397-AF', 61836: '212300979'}
  setting="Enable ACC" value=Off (0x4)
  setting="E

# Compare original openpilot FW matching vs custom fuzzy matcher

In [3]:
from cereal import car
from panda.python.uds import DATA_IDENTIFIER_TYPE
from openpilot.selfdrive.car.fw_query_definitions import EcuAddrSubAddr
from openpilot.selfdrive.car.ford.tests.test_ford import ECU_ADDRESSES

Ecu = car.CarParams.Ecu

FW_ECUS = ECU_ADDRESSES.copy()
FW_ECUS.pop(Ecu.shiftByWire)

ECU_ADDRESS_TO_NAME = {v: k for k, v in ECU_ADDRESSES.items()}

def get_fw_dict(vin: str) -> dict[EcuAddrSubAddr, list[bytes]]:
  try:
    abd = AsBuiltData.from_vin(vin)
  except Exception as e:
    raise RuntimeError(f'Failed to load AsBuiltData for VIN {vin}') from e

  car_fw = {}

  for ecu in FW_ECUS.values():
    if ecu not in abd.ecus:
      continue

    fw = abd.get_identifiers(ecu)[DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER]
    fw_length = 24
    fw = fw.encode()
    fw = (fw + b'\x00' * fw_length)[:fw_length]

    car_fw[(ecu, None)] = [fw]

  return car_fw

for candidate in list(vins)[:3]:
  print(candidate)
  print(get_fw_dict(candidate))
  print()

1FMSK8DH1NGA52944
{(1840, None): [b'M1MC-14D003-AB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (1888, None): [b'L1MC-2D053-KB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (1798, None): [b'LB5T-14F397-AF\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (2016, None): [b'NB5A-14C204-ARC\x00\x00\x00\x00\x00\x00\x00\x00\x00']}

1FMSK8DH7MGC46036
{(1840, None): [b'M1MC-14D003-AB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (1888, None): [b'L1MC-2D053-BJ\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (1798, None): [b'LB5T-14F397-AF\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (2016, None): [b'MB5A-14C204-BUC\x00\x00\x00\x00\x00\x00\x00\x00\x00']}

1FMSK8DH9MGB42406
{(1840, None): [b'L1MC-14D003-AL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (1888, None): [b'L1MC-2D053-BF\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (1798, None): [b'LB5T-14F397-AF\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (2016, None): [b'MB5A-14C204-ARA\x00\x00\x00\x00\x00\x00\x00\x00\x00']}



kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


In [4]:
from cereal import car
from openpilot.selfdrive.car.fw_query_definitions import LiveFwVersions
from openpilot.selfdrive.car.fw_versions import match_fw_to_car_fuzzy as match_op_fuzzy
from openpilot.selfdrive.car.ford.fingerprints import FW_VERSIONS as FORD_FW_VERSIONS
# from openpilot.selfdrive.car.ford.values import match_fw_to_car_fuzzy as match_custom_fuzzy

Ecu = car.CarParams.Ecu
ESSENTIAL_ECUS = [Ecu.engine, Ecu.eps, Ecu.abs, Ecu.fwdRadar, Ecu.fwdCamera, Ecu.vsa]

ECU_NAMES = {v: k for k, v in Ecu.__dict__.items() if isinstance(v, int)}

def match_fw_to_car_exact(live_fw_versions: LiveFwVersions) -> set[str]:
  """Do an exact FW match. Returns all cars that match the given
  FW versions for a list of "essential" ECUs. If an ECU is not considered
  essential the FW version can be missing to get a fingerprint, but if it's present it
  needs to match the database."""
  invalid = set()
  candidates = FORD_FW_VERSIONS

  for candidate, fws in candidates.items():
    for ecu, expected_versions in fws.items():
      ecu_type = ecu[0]
      addr = ecu[1:]

      found_versions = live_fw_versions.get(addr, set())
      if not len(found_versions):
        # Some models can sometimes miss an ecu, or show on two different addresses
        # FIXME: this logic can be improved to be more specific, should require one of the two addresses
        # if candidate in config.non_essential_ecus.get(ecu_type, []):
        #   continue

        # Ignore non essential ecus
        if ecu_type not in ESSENTIAL_ECUS:
          continue

      # Virtual debug ecu doesn't need to match the database
      if ecu_type == Ecu.debug:
        continue

      if not any(found_version in expected_versions for found_version in found_versions):
        invalid.add(candidate)
        break

  return set(candidates.keys()) - invalid


def match_fw_to_car(fw_versions_dict: LiveFwVersions, allow_fuzzy: bool, allow_custom_fuzzy: bool, debug: bool) -> tuple[bool, set[str]]:
  # Attempt to fingerprint using all FW returned from its queries
  matches = match_fw_to_car_exact(fw_versions_dict)
  if len(matches):
    return True, matches

  matches = match_op_fuzzy(fw_versions_dict, log=debug)

  # If specified and no matches so far, fall back to brand's fuzzy fingerprinting function
  # if allow_custom_fuzzy and not len(matches):
  #   matches |= match_custom_fuzzy(fw_versions_dict, FORD_FW_VERSIONS)

  if len(matches):
    return False, matches

  return True, set()


def get_fingerprint(fw_versions_dict: LiveFwVersions, allow_fuzzy=True, allow_custom_fuzzy=False, debug=False) -> tuple[bool, str]:
  exact_match, matches = match_fw_to_car(fw_versions_dict, allow_fuzzy, allow_custom_fuzzy, debug)
  if len(matches) == 0:
    return False, 'mock'
  elif len(matches) == 1:
    return not exact_match, matches.pop()
  else:
    return not exact_match, 'multiple'


def run_fw_test(vins, debug=False):
  df_matches_rows = []
  columns = ['VIN', 'Comment', 'Fingerprint', 'Fuzzy'] #, 'NewFingerprint', 'NewFuzzy']
  for vin in vins:
    fw_versions_dict = get_fw_dict(vin)
    if debug:
      print(f'{vin=}')
      for addr, fws in fw_versions_dict.items():
        ecu = ECU_ADDRESS_TO_NAME[addr[0]]
        print(f'  Ecu.{ECU_NAMES[ecu]}: {fws}')
      print()

    # uses exact/fuzzy matching
    fuzzy, fingerprint = get_fingerprint(fw_versions_dict, debug=debug)

    # uses exact/fuzzy matching + custom match_fw_to_car_fuzzy from fw config
    # new_fuzzy, new_fingerprint = get_fingerprint(fw_versions_dict, allow_custom_fuzzy=True, debug=debug)

    comment = df_vins_csv[df_vins_csv['vin'] == vin].iloc[0]['comment']
    df_matches_rows.append((vin, comment,
                            fingerprint, fuzzy)) #, new_fingerprint, new_fuzzy))

  df_matches = pd.DataFrame(df_matches_rows, columns=columns)
  # df_matches.sort_values(by=['Fingerprint', 'NewFingerprint', 'Fuzzy', 'NewFuzzy'], inplace=True)
  df_matches.sort_values(by=['Fingerprint', 'Fuzzy'], inplace=True)
  return df_matches

In [5]:
def get_factory_acc(row):
  vin = row['VIN']
  abd = AsBuiltData.from_vin(vin)
  if FordEcu.ImageProcessingModuleA not in abd.ecus:
    return 'no camera'
  value = abd.get_setting_value(VehicleSettings.ipma_enable_adaptive_cruise)
  if FordEcu.CruiseControlModule not in abd.ecus:
    return f'{value} (no radar)'
  return value

def get_factory_tja(row):
  vin = row['VIN']
  abd = AsBuiltData.from_vin(vin)
  if FordEcu.ImageProcessingModuleA not in abd.ecus:
    return 'no camera'
  return abd.get_setting_value(VehicleSettings.ipma_enable_traffic_jam_assist)

In [6]:
df_fw_test = run_fw_test(['1FMSK8DH1LGB19569', '1FMSK8FH6LGB17698'], debug=True)
# df_fw_test.drop(columns=['NewFingerprint', 'NewFuzzy'], inplace=True)

df_fw_test['FactoryACC'] = df_fw_test.apply(get_factory_acc, axis=1)
df_fw_test['FactoryTJA'] = df_fw_test.apply(get_factory_tja, axis=1)

df_fw_test

vin='1FMSK8DH1LGB19569'
  Ecu.eps: [b'L1MC-14D003-AK\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00']
  Ecu.abs: [b'L1MC-2D053-BA\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00']
  Ecu.fwdCamera: [b'LB5T-14F397-AE\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00']
  Ecu.engine: [b'LB5A-14C204-BUJ\x00\x00\x00\x00\x00\x00\x00\x00\x00']

vin='1FMSK8FH6LGB17698'
  Ecu.eps: [b'L1MC-14D003-AK\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00']
  Ecu.abs: [b'L1MC-2D053-BA\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00']
  Ecu.fwdRadar: [b'LB5T-14D049-AB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00']
  Ecu.fwdCamera: [b'LB5T-14F397-AE\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00']
  Ecu.engine: [b'LB5A-14C204-BUJ\x00\x00\x00\x00\x00\x00\x00\x00\x00']



,VIN,Comment,Fingerprint,Fuzzy,FactoryACC,FactoryTJA
1,1FMSK8FH6LGB17698,2020 Ford Explorer Limited,FORD EXPLORER 6TH GEN,False,RadarFusion,On
0,1FMSK8DH1LGB19569,2020 Ford Explorer XLT,mock,False,Off (no radar),Off


In [7]:
def trim_vin(row):
  return row['VIN'][0:12] + 'X' * 6


# vins = set(df_vin_values['details', 'vin'])
df_fw_test = run_fw_test(vins)

df_fw_test['FactoryACC'] = df_fw_test.apply(get_factory_acc, axis=1)
df_fw_test['FactoryTJA'] = df_fw_test.apply(get_factory_tja, axis=1)

df_fw_test['VIN'] = df_fw_test.apply(trim_vin, axis=1)

print(df_fw_test.shape)
print(df_fw_test.to_string())

Unknown ECU address: 1808 (0x710)
0x710 {61706: 'NB3V-14E098-AB', 61712: 'DSNB3V-14E090-AB', 61713: 'NB3V-14E099-AA', 61715: 'NB3V-14E090-AB', 61832: 'NB3V-14E093-AB', 61836: '0000000002363440', 61920: 'PXNB3V-14E090-AA'}
Unknown ECU address: 1808 (0x710)
0x710 {61706: 'NB3C-14E098-BB', 61712: 'DSNB3V-14E090-AB', 61713: 'NB3V-14E099-AA', 61715: 'NB3V-14E090-AB', 61832: 'NB3V-14E093-AB', 61836: '0000000002312882', 61920: 'PXNB3V-14E090-AA'}
Unknown ECU address: 1808 (0x710)
0x710 {61706: 'NB3C-14E098-BC', 61712: 'DSNB3V-14E090-AB', 61713: 'NB3V-14E099-AA', 61715: 'NB3V-14E090-AB', 61832: 'NB3V-14E093-AB', 61836: '0000000002417620', 61920: 'PXNB3V-14E090-AA'}
Unknown ECU address: 1808 (0x710)
0x710 {61706: 'NB3C-14E098-BC', 61712: 'DSNB3V-14E090-AB', 61713: 'NB3V-14E099-AA', 61715: 'NB3V-14E090-AB', 61832: 'NB3V-14E093-AB', 61836: '0000000002400325', 61920: 'PXNB3V-14E090-AA'}
Unknown ECU address: 2002 (0x7d2)
0x7d2 {61712: 'DSNL38-10D678-AG', 61713: 'NL38-10D686-AC', 61715: 'NL38-10D678

In [8]:
# def get_result(row):
#   if row['Fingerprint'] == 'mock' and row['NewFingerprint'] == 'mock':
#     return 'mock -> mock'
#   elif row['Fingerprint'] == 'mock':
#     return 'mock -> fp'
#   elif row['NewFingerprint'] == 'mock':
#     return 'fp -> mock'
#   else:
#     return 'fp -> fp'

def trim_vin(row):
  return row['VIN'][0:12] + 'X' * 6


# vins = set(df_vin_values['details', 'vin'])
df_fw_test = run_fw_test(vins)

df_fw_test['FactoryACC'] = df_fw_test.apply(get_factory_acc, axis=1)
df_fw_test['FactoryTJA'] = df_fw_test.apply(get_factory_tja, axis=1)

# df_fw_test['Result'] = df_fw_test.apply(get_result, axis=1)
df_fw_test['VIN'] = df_fw_test.apply(trim_vin, axis=1)

print(df_fw_test.shape)
print(df_fw_test.to_string())

(845, 6)
                    VIN                                      Comment                  Fingerprint  Fuzzy              FactoryACC FactoryTJA
64   1FMCU0G6XMUAXXXXXX                          2021 Ford Escape SE          FORD ESCAPE 4TH GEN  False             RadarFusion         On
67   1FMCU0G62MUAXXXXXX                          2021 Ford Escape SE          FORD ESCAPE 4TH GEN  False             RadarFusion         On
70   1FMCU0G61LUBXXXXXX                          2020 Ford Escape SE          FORD ESCAPE 4TH GEN  False             RadarFusion        Off
71   1FMCU0H69LUBXXXXXX                         2020 Ford Escape SEL          FORD ESCAPE 4TH GEN  False             RadarFusion        Off
166  1FMCU0G6XLUAXXXXXX                          2020 Ford Escape SE          FORD ESCAPE 4TH GEN  False             RadarFusion        Off
200  1FMCU9H67NUBXXXXXX                         2022 Ford Escape SEL          FORD ESCAPE 4TH GEN  False             RadarFusion         On
207  1FMCU0

In [9]:
# statistics
df_fw_test_results = df_fw_test[['Result']].copy()
df_fw_test_results['Count'] = 1
df_fw_test_results = df_fw_test_results.groupby(['Result']).count()
df_fw_test_results.reset_index(inplace=True)
df_fw_test_results.columns = ['Result', 'Count']
df_fw_test_results['%'] = (df_fw_test_results['Count'] / df_fw_test_results['Count'].sum() * 100).round(1)
df_fw_test_results

KeyError: "None of [Index(['Result'], dtype='object')] are in the [columns]"

In [ ]:
from openpilot.selfdrive.car.ford.values import CAR

ALL_ECUS = {k: v for k, v in Ecu.__dict__.items() if isinstance(v, int)}
ECU_LOOKUP = {addr: ecu for ecu, addr in ALL_ECUS.items()}
FW_ECUS_BY_ADDR = {addr: ecu for ecu, addr in FW_ECUS.items()}

def check_fw_database(df):
  for _, row in df.iterrows():
    vin = row['VIN']
    print(row[['VIN', 'Model', 'ModelYear']].to_string())
    print('FW:')

    fw_rows = []
    for addr, fws in get_fw_dict(vin).items():
      ecu = FW_ECUS_BY_ADDR[addr[0]]
      fw = fws[0]

      known_fw = fw in FORD_FW_VERSIONS[CAR.EXPLORER_MK6][(ecu, *addr)]

      fw_rows.append((ECU_LOOKUP[ecu], fw, known_fw))
    df_fw = pd.DataFrame(fw_rows, columns=['ECU', 'FW', 'In Database'])
    print(df_fw.to_string())
    print()

In [ ]:
def is_changed_from_mock(row):
  return row['ExistingFingerprint'] == 'mock' and row['Changed']

df_new_fps = df_fw_test[df_fw_test.apply(is_changed_from_mock, axis=1)].copy()
check_fw_database(df_new_fps)

In [ ]:
def is_not_changed_from_mock(row):
  return row['ExistingFingerprint'] == 'mock' and not row['Changed']

df_mock_unchanged = df_fw_test[df_fw_test.apply(is_not_changed_from_mock, axis=1)].copy()
check_fw_database(df_mock_unchanged)